# Problem 03-10

## Solution

In [9]:
def key(x):
    full_stop = '.'
    alpha_small = 'abcdefghijklmnopqrstuvwxyz'
    alpha_big = alpha_small.upper()
    num = '0123456789'
    punct = '!"#$%&()*+,-/:;<=>?@[\]^_{|}~'
    letters_small = list(alpha_small)
    letters_big = list(alpha_big)
    punct = list(punct)

    indices_small = list(range(0, len(alpha_small)))
    indices_big = list(range(0, len(alpha_big)))
    indices_punct = list(range(0, len(punct)))

    d_small = dict(zip(letters_small, indices_small))
    d_big = dict(zip(letters_big, indices_big))
    d_punct = dict(zip(punct, indices_punct))


    repres = []  # тут репрезентация строки в виде кортежа вида ( int <класс объекта> , int <ранг объекта в классе> )
    found_num = ''  # чтоб запоминать число
    for let in x:
        if let == full_stop:
            if found_num != '':
                repres.append((3, int(found_num)))
                found_num = ''  # если число в начале или в середине, то мы идем пока не встретим букву и тогда аппендим число и удаляем его из found_num
            repres.append((0, 0))
        if let in alpha_small:
            if found_num != '':
                repres.append((1, int(found_num)))
                found_num = ''  # если число в начале или в середине, то мы идем пока не встретим букву и тогда аппендим число и удаляем его из found_num
            repres.append((3, d_small[let]))
        elif let in alpha_big:
            if found_num != '':
                repres.append((1, int(found_num)))
                found_num = ''  # если число в начале или в середине, то мы идем пока не встретим букву и тогда аппендим число и удаляем его из found_num
            repres.append((2, d_big[let]))
        elif let in punct:
            if found_num != '':
                repres.append((1, int(found_num)))
                found_num = ''  # если число в начале или в середине, то мы идем пока не встретим букву и тогда аппендим число и удаляем его из found_num
            repres.append((4, d_punct[let]))
        elif let in num:
            found_num += let
    if found_num != '':  # если число в конце (потому что аппенд был только если мы встретили букву дальше)
        repres.append((1, int(found_num)))
    return repres

## Strategies

In [5]:
from hypothesis import given, note
from hypothesis.strategies import integers, lists, text, composite, one_of

alpha = 'abcxyzABCXYZ'
digits = '0123456789'
punct = '!"#$%&\'()*+,-./:;?@[\\]^_`{|}~'

part = one_of(
    text(alphabet=alpha, min_size=1, max_size=1),
    text(alphabet=digits, min_size=1, max_size=4),
    text(alphabet=punct, min_size=1, max_size=1)
)

@composite
def strings(draw):
    return draw(lists(lists(elements=part, min_size=3).map(lambda x: ''.join(x)),
                      min_size=20, max_size=100))

# Test

In [6]:
def bash_sort(array):
    import subprocess
    
    data = ("\n".join(map(str, array))).encode('utf-8')
    
    proc = subprocess.Popen(['/usr/bin/sort', '-n'],
                            stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    try:
        outs, errs = proc.communicate(data, timeout=15)
    except subprocess.TimeoutExpired:
        proc.kill()
        raise

    return outs.decode('utf-8').strip().split("\n")

bash_sort(["1000", '1'])


@given(data=strings())
def test_bash_sort(data):
    assert sorted(data, key=key) == bash_sort(data)

test_bash_sort()
print('ok')

Falsifying example: test_bash_sort(data=['aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 '1aa'])


AssertionError: 

In [8]:
d = ['aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 '1aa']

sorted(d, key=key)
bash_sort(['a', 'b'])

['1aa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa',
 'aaa']